# Time series
75033710 rows

13713 unique time series taken in 12603 stations from 17-10-2000 to 22-03-2018

In [1]:
import pyarrow.feather as feather
import pandas as pd
import numpy as np
from scipy.io import netcdf
import matplotlib.pyplot as plt 

In [ ]:
time_series_df = feather.read_feather('dataset/original_data_as_feather')
time_series_df.head()

In [ ]:
print(len(time_series_df))
print(len(time_series_df["proj_id"].unique()))
print(len(time_series_df["name"].unique()))
print(time_series_df.isnull().sum())

In [ ]:
#pd.DataFrame(data=(time_series_df.loc[time_series_df.index[0]],time_series_df.loc[time_series_df.index[-1]]))

In [ ]:
t_s_1 = time_series_df[time_series_df["proj_id"]=="BB_25470023"]
plt.plot(t_s_1["date"],t_s_1["groundwater_level"])

In [ ]:
t_s_2 = time_series_df[time_series_df["proj_id"]==time_series_df["proj_id"].unique()[5002]]
plt.plot(t_s_2["date"],t_s_2["groundwater_level"])

In [ ]:
len(t_s_2["date"].unique())

In [ ]:
t_s_2["date"].unique()[10:20]

# filter time series to be from 1995 to 2012
## and drop unnedded columns

In [1]:
###############

# metadata: 
respective observation wells ( stations),

related to the equipment used in the measurement e.g. coordinates, filter depth

13492 rows, 15 feature
latitude and longitude of stations are taken from here

In [ ]:
meta_df = pd.read_excel("dataset/metadata_timeseries_LongLat.xlsx")
meta_df.head()

In [ ]:
meta_df.shape

In [ ]:
len(meta_df["y_lat"].unique())

In [ ]:
plt.scatter(meta_df["x_long"],meta_df["y_lat"])
plt.show()

 ### predictors ( temperature and precipitation) are matched to the station lon lat acording these meta data

# temperature: as predictor

Daily means of hourly grids of air temperature for whole Germany (project TRY Advancement)

there are 215 NC files, each file represents a month of dailly temperature from 01-1995 tp 11-2012



In [ ]:
# extract relevant temp to T.S from an nc file 

In [ ]:
# each nc file has temperature across whole germany
dir_01_2005 = "dataset/project_try_dwd/data/"+'daily_Project_TRY_air_temperature_mean__TT_200511_daymean.nc'
temp_01_2005 = netcdf.NetCDFFile(dir_01_2005,'r')

In [ ]:
temp_01_2005.variables

In [ ]:
# long and lat where temp was measured 
x_long = temp_01_2005.variables["lon"][:].flatten()
y_lat = temp_01_2005.variables["lat"][:].flatten()


In [ ]:
938*720

In [ ]:
# create df to store the temp T.S for each station
station_temp_df =meta_df[["proj_id","x_long","y_lat"]]
station_temp_df.head()

In [ ]:
# extract the indices of the temp - matching the locations between weather stations ans bgr stations

def temp_idx(df):
    # compute the temp_index according the closest long_lat
    long = df["x_long"]
    lat  = df["y_lat"]
    # compute the closest temp_long to the station- all occurances
    a = np.absolute(x_long - long)
    # compute the closest temp_lat to the station- all occurances
    b = np.absolute(y_lat - lat)

    if np.min(a) <= np.min(b):
        min_long = np.argwhere(a == np.min(a))
        idx = [int(i) for i in min_long if b[i] == np.min(b[min_long])][0]

    else:
        min_lat = np.argwhere(b == np.min(b))
        idx = [int(i) for i in min_lat if a[i] == np.min(a[min_lat])][0]
    
    return idx



In [ ]:
# the indicies computed once and stored 
#station_temp_df["idx"] = station_temp_df.apply(temp_idx,axis=1) 
# station_temp_df.head()
#station_temp_df = station_temp_df.rename(columns={"Temp_20050103":"idx"})
# save in temporary dataset file for later use
#station_temp_df.to_csv("dataset/temporary/station_temp_df.csv", index=False)

In [ ]:
temp_df = pd.read_csv("dataset/temporary/station_temp_df.csv")
temp_df.head()

In [ ]:
t_s_chunklist = []
filtered_t_s = pd.read_csv(".csv", chunksize = 10e6)


In [2]:
df = pd.read_csv("filtered_t_s.csv")



In [ ]:
df.describe(include ="all")

In [5]:
df[df["proj_id"]=="BB_36459823"]

,proj_id,date,groundwater_level
1074073,BB_36459823,1995-01-08 00:00:00+00:00,35.93
1074074,BB_36459823,1995-01-15 00:00:00+00:00,35.92
1074075,BB_36459823,1995-01-22 00:00:00+00:00,35.97
1074076,BB_36459823,1995-02-01 00:00:00+00:00,35.97
1074077,BB_36459823,1995-02-08 00:00:00+00:00,36.03
...,...,...,...
1074798,BB_36459823,2012-11-22 00:00:00+00:00,36.12
1074799,BB_36459823,2012-12-01 00:00:00+00:00,36.13
1074800,BB_36459823,2012-12-08 00:00:00+00:00,36.12
1074801,BB_36459823,2012-12-15 00:00:00+00:00,36.10


In [ ]:
dates = df["date"]
dates

In [ ]:
df["day"] = df["date"].apply(lambda d:"".join(str(pd.to_datetime(str(d)).date()).split("-")))

In [ ]:
t_s_1 = df[df["proj_id"]=="BB_25470023"]

In [ ]:
plt.plot(t_s_1["date"],t_s_1["groundwater_level"])

In [ ]:
# pathes to all nc files
from os import walk
folder_path =  "dataset/project_try_dwd/data/"
files= []
for (dirpath, dirnames, filenames) in walk(folder_path):
    files.extend(filenames)
    break

In [ ]:
unique_dates = t_s_1["date"].unique() # for t_s_1
unique_dates = ["".join(str(pd.to_datetime(str(d)).date()).split("-")) for d in unique_dates]

In [ ]:
len(unique_dates)

In [ ]:
for file in files:
    month = folder_path + file
    nc_file = netcdf.NetCDFFile(month,'r')
    days = nc_file.variables["datum"][:]
    days = [str(x)[:-4] for x in days]
    for d, day in enumerate(days):
        if day in unique_dates:
            temp_df["t"+str(day)] = nc_file.variables["temperature"][d,:,:].flatten()[temp_df["idx"]]


In [ ]:
dirc = "dataset/project_try_dwd/data/"+files[0]
dirc

In [ ]:
plt.figure(figsize=(50,15))
t_s_1.plot(color='blue', grid=True)
plt.legend(loc='upper left')
plt.xlabel('t s temp_BB_25470023 ')
temp_BB_254700230.plot(color='red', grid=True, secondary_y=True)
plt.legend(loc='upper right')
plt.xlabel('temp temp_BB_25470023 ')
plt.show()